# IAQF 2021 Code Collection

In [ ]:
PROBLEM_NO = 1
PERIOD_NO = 'basic'

In [ ]:
# Import libraries

import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sbn

import sklearn
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingClassifier,GradientBoostingRegressor
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import accuracy_score

from xgboost import XGBClassifier
from xgboost import XGBRegressor

import tensorflow as tf
from tensorflow import keras

from openpyxl import load_workbook

In [ ]:
OS = 'Mac'  # OS is either 'Mac' or 'Windows'
sep = '/' if OS == 'Mac' else '\\'
path = "source_data{}{}.csv"

PROBLEM_PATH_DICT = {
    1: {
        'basic': path.format(sep, "crude_gas_et_df"),
        'et': path.format(sep, "crude_gas_et_df"),
        'p1': path.format(sep, "crude_gas_p1_df"),
        'p1.2': path.format(sep, "crude_gas_p2_df"),
        'p2': path.format(sep, "crude_gas_p2_df")
    },  # crude oil price vs. gasoline price
    2: {
        'basic': path.format(sep, "cf_gas_et_df"),
        'et': path.format(sep, "cf_gas_et_df"),
        'p1': path.format(sep, "cf_gas_p1_df"),
        'p1.2': path.format(sep, "cf_gas_p2_df"),
        'p2': path.format(sep, "cf_gas_p2_df"),
    },  # crude oil futures price vs. gasoline price
    3: {
        'basic': path.format(sep, "crude_co2_et_df"),
        'et': path.format(sep, "crude_co2_et_df"),
        'p1': path.format(sep, "crude_co2_p1_df"),
        'p1.2': path.format(sep, "crude_co2_p2_df"),
        'p2': path.format(sep, "crude_co2_p2_df")
    }  # crude oil price vs. co2
}

PROBLEM_VAR_DICT = {
    1: ('crude_oil', 'gasoline'),
    2: ('oil_futures', 'gasoline'),
    3: ('crude_oil', 'co2')
}

ADDITIONAL_VAR_DICT = {
    1: {
        'basic': [],
        'et': ['cpi','us_pop','public_roads','us_urban_pop','oil_production','auto_sales','us_real_gdp'],
        'p1': ['cpi','us_pop','public_roads','us_urban_pop','oil_production','auto_sales','us_real_gdp'],
        'p1.2': ['cpi','us_pop','public_roads','us_urban_pop','oil_production','auto_sales','us_real_gdp'],
        'p2': ['cpi','us_pop','public_roads','us_urban_pop','oil_production','djus_auto_index','auto_sales','us_real_gdp','usd_mex_exrate']
        
    },
    2: {
        'basic': [],
        'et': ['cpi','us_pop','public_roads','us_urban_pop','oil_production','auto_sales','us_real_gdp'],
        'p1': ['cpi','us_pop','public_roads','us_urban_pop','oil_production','auto_sales','us_real_gdp'],
        'p1.2': ['cpi','us_pop','public_roads','us_urban_pop','oil_production','auto_sales','us_real_gdp'],
        'p2': ['cpi','us_pop','public_roads','us_urban_pop','oil_production','djus_auto_index','auto_sales','us_real_gdp','usd_mex_exrate']

    },
    3: {
        'basic': [],
        'et': ['cpi','coal_consumption','ngas_consumption','us_pop','oil_production','tree_cover_loss','us_real_gdp'],
        'p1': ['cpi','coal_consumption','ngas_consumption','us_pop','oil_production','tree_cover_loss','us_real_gdp'],
        'p1.2': ['cpi','coal_consumption','ngas_consumption','us_pop','oil_production','tree_cover_loss','us_real_gdp'],
        'p2': ['cpi','coal_consumption','ngas_consumption','us_pop','oil_production','cattle_index','tree_cover_loss','us_real_gdp','usd_mex_exrate']
    }
}






sheetname = str(PROBLEM_NO) + PERIOD_NO

X_MAIN_COL, Y_COL = PROBLEM_VAR_DICT[PROBLEM_NO]
X_MAIN_CHG_COL = X_MAIN_COL + '_chg'
Y_CHG_COL = Y_COL + '_chg'
EXTRA_VARS = ADDITIONAL_VAR_DICT[PROBLEM_NO][PERIOD_NO]

TRAINING_RATIO_DEFAULT = 0.7
VALIDATION_RATIO_DEFAULT = 0.15
TEST_RATIO_DEFAULT = 0.15

REGRESSOR_ACCURACY_DICT = {}
CLASSIFIER_ACCURACY_DICT = {}

REGRESSOR_COEF_DICT = {}
CLASSIFIER_COEF_DICT = {}

df = pd.read_csv(PROBLEM_PATH_DICT[PROBLEM_NO][PERIOD_NO])
df

## Creating independent and dependent variables

In [ ]:
# calculate independent variables
periods = (1,2,3,5,10)
for i in periods:
    df [X_MAIN_CHG_COL + f'_{i}'] = df[X_MAIN_COL].pct_change(periods=i)

# calculate dependent variable
df[Y_CHG_COL] = df[Y_COL].pct_change(periods=1)

# eliminate the empty rows
df = df[11:]
df

## Seperating training, validation, and test datasets

In [ ]:
# create list of independent variable's name
indep_vars = [X_MAIN_CHG_COL + f'_{i}' for i in periods] + EXTRA_VARS
num_var =len(indep_vars)

# extract the values to X
X = df[[X_MAIN_COL] + indep_vars].values
X

In [ ]:
# create y for regressor, y_c for classifier
y = df[Y_CHG_COL]
y_c = (df[Y_CHG_COL] > 0).values.astype('int')

# set training, validation, and test criterias
training_ratio = TRAINING_RATIO_DEFAULT
validation_ratio = VALIDATION_RATIO_DEFAULT
test_ratio = TEST_RATIO_DEFAULT

# check correctness of X and y
X.shape, y.shape, y_c.shape

In [ ]:
# create training, validation, and test sets for regressor
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_ratio, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=validation_ratio/(validation_ratio+training_ratio), random_state=42)

print("X_train shape:\t", X_train.shape)
print("X_test shape:\t", X_test.shape)
print("X_val shape:\t", X_val.shape)
print("y_train shape:\t", y_train.shape)
print("y_val shape:\t", y_val.shape)
print("y_test shape:\t", y_test.shape)

In [ ]:
# create training, validation, and test sets for classifier
X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(X, y_c, test_size=test_ratio, random_state=42)
X_train_c, X_val_c, y_train_c, y_val_c = train_test_split(X_train_c, y_train_c, test_size=validation_ratio/(validation_ratio+training_ratio), random_state=42)

print("X_train_c shape:", X_train_c.shape)
print("X_test_c shape:\t", X_test_c.shape)
print("X_val_c shape:\t", X_val_c.shape)
print("y_train_c shape:", y_train_c.shape)
print("y_val_c shape:\t", y_val_c.shape)
print("y_test_c shape:\t", y_test_c.shape)

## Principal Component Analysis (PCA)

In [ ]:
# normalization for regressor set
for i in range(X_train.shape[1]):
    X_train_mean = X_train[:, i]. mean()
    X_train_std = X_train[:, i].std()
    X_train[:, i] = (X_train[:,i] - X_train_mean) / X_train_std
    X_test[:,i] = (X_test[:,i] - X_train_mean) / X_train_std
    X_val[:, i] = (X_val[:,i] - X_train_mean) / X_train_std

print("X_train shape:\t", X_train.shape)
print("X_val shape:\t", X_val.shape)

In [ ]:
# normalization for classifier set
for i in range(X_train_c.shape[1]):
    X_train_c_mean = X_train_c[:, i]. mean()
    X_train_c_std = X_train_c[:, i].std()
    X_train_c[:, i] = (X_train_c[:,i] - X_train_c_mean) / X_train_c_std
    X_test_c[:,i] = (X_test_c[:,i] - X_train_c_mean) / X_train_c_std
    X_val_c[:, i] = (X_val_c[:,i] - X_train_c_mean) / X_train_c_std

print("X_train_c shape:", X_train_c.shape)
print("X_val_c shape:\t", X_val_c.shape)

In [ ]:
# PCA for regressor
# set PCA value
pca = PCA(0.95)

# fit PCA training set
pca.fit(X_train)

# apply transform to training and test set
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)
X_val_pca = pca.transform(X_val)

print("X_train_pca shape:\t", X_train_pca.shape)
print("X_test_pca shape:\t", X_test_pca.shape)
print("X_val_pca shape:\t", X_val_pca.shape)
print("y_train shape:\t", y_train.shape)
print("y_val shape:\t", y_val.shape)
print("y_test shape:\t", y_test.shape)

In [ ]:
# PCA for classifier
# apply transform to training and test set
X_train_c_pca = pca.transform(X_train_c)
X_test_c_pca = pca.transform(X_test_c)
X_val_c_pca = pca.transform(X_val_c)

print("X_train_c_pca shape:\t", X_train_c_pca.shape)
print("X_test_c_pca shape:\t", X_test_c_pca.shape)
print("X_val_c_pca shape:\t", X_val_c_pca.shape)
print("y_train_c shape:", y_train_c.shape)
print("y_val_c shape:\t", y_val_c.shape)
print("y_test_c shape:\t", y_test_c.shape)

In [ ]:
num_var = X_train_c_pca.shape[1]
num_var

## Multiple linear regression

In [ ]:
multiple_reg = LinearRegression().fit(X_train_pca, y_train)
score_multiple_reg = multiple_reg.score(X_test_pca, y_test)

print('score_multiple_reg:', "%.4f" % score_multiple_reg)

In [ ]:
REGRESSOR_COEF_DICT['multi_linear'] = multiple_reg.coef_
REGRESSOR_ACCURACY_DICT['multi_linear'] = score_multiple_reg

## Random forest classifier

In [ ]:
# testing for different max depth
rfc = dict()
for i in range (1, 15):
    clf = RandomForestClassifier(max_depth=i, random_state=0).fit(X_train_c_pca, y_train_c)
    y_pred = clf.predict(X_val_c_pca)
    score_clf = accuracy_score(y_val_c, y_pred)
    rfc[i] = score_clf

max_depth = max(rfc, key=rfc.get)
print('max depth:', max_depth)

In [ ]:
# choose max_depth = 4
rf_clf = RandomForestClassifier(max_depth=int(max_depth), random_state=0).fit(X_train_c_pca, y_train_c)
score_rf_clf = rf_clf.score(X_test_c_pca, y_test_c)
print('score_rf_clf:', "%.4f" % score_rf_clf)

In [ ]:
CLASSIFIER_COEF_DICT['random_forest'] = rf_clf.feature_importances_
CLASSIFIER_ACCURACY_DICT['random_forest'] = score_rf_clf

## Random forest regressor

In [ ]:
rf_reg = RandomForestRegressor().fit(X_train_pca, y_train)
score_rf_reg = rf_reg.score(X_test_pca, y_test)
print('score_rf_reg:', "%.4f" % score_rf_reg)

In [ ]:
REGRESSOR_COEF_DICT['random_forest'] = rf_reg.feature_importances_
REGRESSOR_ACCURACY_DICT['random_forest'] = score_rf_reg

## Logistic regression

In [ ]:
logistic_reg = LogisticRegression(penalty = 'l1', solver='liblinear', random_state=0).fit(X_train_c_pca, y_train_c)
score_logistic = logistic_reg.score(X_test_c_pca, y_test_c)
print('score_logistic:', "%.4f" % score_logistic)

In [ ]:
CLASSIFIER_COEF_DICT['logistic'] = logistic_reg.coef_[0]
CLASSIFIER_ACCURACY_DICT['logistic'] = score_logistic

## Gradient boosting classifier

In [ ]:
gra_clf = GradientBoostingClassifier(n_estimators = 200, learning_rate=0.50, max_depth=5, random_state=0).fit(X_train_c_pca, y_train_c)
score_gb_clf = gra_clf.score(X_test_c_pca, y_test_c)
score_gb_clf

In [ ]:
CLASSIFIER_COEF_DICT['gradient_boosting'] = gra_clf.feature_importances_
CLASSIFIER_ACCURACY_DICT['gradient_boosting'] = score_gb_clf

## Gradient boosting regressor

In [ ]:
gra_reg = GradientBoostingRegressor(random_state=0).fit(X_train_pca, y_train)
score_gb_reg = gra_reg.score(X_test_pca, y_test)
score_gb_reg

In [ ]:
REGRESSOR_COEF_DICT['gradient_boosting'] = gra_reg.feature_importances_
REGRESSOR_ACCURACY_DICT['gradient_boosting'] = score_gb_reg

## XGBoost classifier

In [ ]:
xgb_clf = XGBClassifier(use_label_encoder=False).fit(X_train_c_pca, y_train_c)
score_xgb_clf = xgb_clf.score(X_test_c_pca, y_test_c)
score_xgb_clf

In [ ]:
CLASSIFIER_COEF_DICT['xgboost'] = xgb_clf.feature_importances_
CLASSIFIER_ACCURACY_DICT['xgboost'] = score_xgb_clf

## XGBoost regressor

In [ ]:
xgb_reg = XGBRegressor(random_state = 0).fit(X_train_pca, y_train)
score_xgb_reg = xgb_reg.score(X_test_pca, y_test)
score_xgb_reg

In [ ]:
REGRESSOR_COEF_DICT['xgboost'] = xgb_reg.feature_importances_
REGRESSOR_ACCURACY_DICT['xgboost'] = score_xgb_reg

## AdaBoost classifier

In [ ]:
adb_clf = AdaBoostClassifier().fit(X_train_c_pca, y_train_c)
score_adb_clf = adb_clf.score(X_test_c_pca, y_test_c)
score_adb_clf

In [ ]:
CLASSIFIER_COEF_DICT['adaboost'] = adb_clf.feature_importances_
CLASSIFIER_ACCURACY_DICT['adaboost'] = score_adb_clf

## AdaBoost regressor

In [ ]:
adb_reg = AdaBoostRegressor().fit(X_train_pca, y_train)
score_adb_reg = adb_reg.score(X_test_pca, y_test)
score_adb_reg

In [ ]:
REGRESSOR_COEF_DICT['adaboost'] = adb_reg.feature_importances_
REGRESSOR_ACCURACY_DICT['adaboost'] = score_adb_reg

## Support vector classification

In [ ]:
# Try different kernels
kernels = ('linear', 'poly', 'rbf')

for k in kernels:
    svm_clf = SVC(kernel=k).fit(X_train_c_pca, y_train_c)
    y_pred = svm_clf.predict(X_test_c_pca)
    vars()[f'score_svc_{k}']= accuracy_score(y_test_c, y_pred)
    
    if k == 'linear':
        CLASSIFIER_COEF_DICT['svc_linear'] = svm_clf.coef_[0]
    
print(score_svc_linear)
print(score_svc_poly)
print(score_svc_rbf)

CLASSIFIER_ACCURACY_DICT['svc_linear'] = score_svc_linear
CLASSIFIER_ACCURACY_DICT['svc_poly'] = score_svc_poly
CLASSIFIER_ACCURACY_DICT['svc_rbf'] = score_svc_rbf

## Gaussian Naive Bayes

In [ ]:
gnb_clf = GaussianNB().fit(X_train_c_pca, y_train_c)
score_gnb = gnb_clf.score(X_test_c_pca, y_test_c)
print(score_gnb)

In [ ]:
CLASSIFIER_ACCURACY_DICT['GaussianNB'] = score_gnb

## Neuro networks

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.Dense(10, input_dim=num_var, activation='relu'))
model.add(keras.layers.Dense(5, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(X_train_c_pca, y_train_c, epochs=50, validation_data=(X_val_c_pca, y_val_c))

In [ ]:
loss, score_neuro = model.evaluate(X_test_c_pca, y_test_c)
print(score_neuro)

In [ ]:
CLASSIFIER_ACCURACY_DICT['neuro'] = score_neuro

## Summary

In [ ]:
reg_df = pd.DataFrame.from_dict(REGRESSOR_ACCURACY_DICT , orient='index', columns=['Score'])
reg_df.index.name = 'Regression models'
#reg_df.sort_values(by=['Score'], ascending=False)
reg_df

In [ ]:
clf_df = pd.DataFrame.from_dict(CLASSIFIER_ACCURACY_DICT, orient='index',columns=['Score'])
clf_df.index.name = 'Classifier models'
#clf_df.sort_values(by=['Score'], ascending=False)
clf_df

In [ ]:
reg_coef_df = pd.DataFrame.from_dict(REGRESSOR_COEF_DICT, orient='index')
reg_coef_df

In [ ]:
clf_coef_df = pd.DataFrame.from_dict(CLASSIFIER_COEF_DICT, orient='index')
clf_coef_df

In [ ]:
# Export
book = load_workbook('Results.xlsx')
writer = pd.ExcelWriter('Results.xlsx', engine='openpyxl')
writer.book = book

writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
reg_df.to_excel(writer, sheet_name='%s reg score' % sheetname)
clf_df.to_excel(writer, sheet_name='%s clf score' % sheetname)
reg_coef_df.to_excel(writer, sheet_name='%s reg coef' % sheetname)
clf_coef_df.to_excel(writer, sheet_name='%s clf coef' % sheetname)

writer.save()
#reg_df.to_csv('reg_df.csv')
#clf_df.to_csv('clf_df.csv')
#reg_coef_df.to_csv('reg_coef_df.csv')
#clf_coef_df.to_csv('clf_coef_df.csv')